In [ ]:

from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import os
os.chdir("drive/My Drive/Ben_Boyd_MSc_Project/Data")


Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
from sklearn import datasets, preprocessing
from IPython.display import clear_output
import numpy as onp
import jax.numpy as np # JAX is supposed to have an API that closely resemble NumPy's
from jax import grad, jit, vmap, value_and_grad
from jax import random
import jax.nn as nn
import copy
%matplotlib inline

from jax.experimental.stax import (Dense, Tanh, Flatten, Relu, LogSoftmax, Softmax, Exp,Sigmoid,Softplus,LeakyRelu)

import matplotlib.pyplot as plt # creating visualizations

from matplotlib import cm
from matplotlib.ticker import LinearLocator

import pickle

from astropy.table import Table

from astropy import table
from astropy.io import fits,ascii,votable
from astropy import units as u 
from astropy import constants as const
from astropy import table

In [ ]:

from jax import nn
from jax.experimental import stax, optimizers

In [ ]:
import jax.numpy as np
import numpy as onp
from jax import nn, ops, random
from jax.experimental import stax

DEFAULT_MIN_BIN_WIDTH = 1e-3
DEFAULT_MIN_BIN_HEIGHT = 1e-3
DEFAULT_MIN_DERIVATIVE = 1e-3


def searchsorted(bin_locations, inputs, eps=1e-6):
    bin_locations = bin_locations.at[..., -1].add(eps)  # bin_locations[..., -1] += eps
    return np.sum(inputs[..., None] >= bin_locations, axis=-1) - 1


def unconstrained_RQS(
    inputs,
    unnormalized_widths,
    unnormalized_heights,
    unnormalized_derivatives,
    inverse=False,
    tail_bound=1.0,
    min_bin_width=DEFAULT_MIN_BIN_WIDTH,
    min_bin_height=DEFAULT_MIN_BIN_HEIGHT,
    min_derivative=DEFAULT_MIN_DERIVATIVE,
):
    #inside_interval_mask = (inputs >= -tail_bound) & (inputs <= tail_bound)
    #outside_interval_mask = ~inside_interval_mask
  

    outputs = np.zeros_like(inputs)
    logabsdet = np.ones_like(inputs)

    unnormalized_derivatives = np.pad(
        unnormalized_derivatives, [(0, 0)] * (len(unnormalized_derivatives.shape) - 1) + [(1, 1)]
    )
    constant = np.log(np.exp(1 - min_derivative) - 1)
    unnormalized_derivatives = unnormalized_derivatives.at[..., 0].set(constant)  # unnormalized_derivatives[..., 0] = constant
    unnormalized_derivatives = unnormalized_derivatives.at[..., -1].set(constant)  # unnormalized_derivatives[..., -1] = constant

        
    logabsdet = np.where(inputs<-tail_bound,0,logabsdet)
    logabsdet = np.where(inputs> tail_bound , 0, logabsdet)
    
    outputs = np.where(logabsdet==0,inputs,outputs)






    #outputs = outputs.at[outside_interval_mask].set(inputs[outside_interval_mask])
    # outputs[outside_interval_mask] = inputs[outside_interval_mask]
    #logabsdet = logabsdet.at[outside_interval_mask].set(0)
     # logabsdet[outside_interval_mask] = 0

    outs, logdets = RQS(
        inputs=inputs,
        unnormalized_widths=unnormalized_widths,
        unnormalized_heights=unnormalized_heights,
        unnormalized_derivatives=unnormalized_derivatives,
        inverse=inverse,
        left=-tail_bound,
        right=tail_bound,
        bottom=-tail_bound,
        top=tail_bound,
        min_bin_width=min_bin_width,
        min_bin_height=min_bin_height,
        min_derivative=min_derivative,
    )

    outputs = np.where(logabsdet==1,outs,outputs)  # outputs[inside_interval_mask] = outs
    logabsdet = np.where(logabsdet==1,logdets,logabsdet) # logabsdet[inside_interval_mask] = logdets

    return outputs, logabsdet


def RQS(
    inputs,
    unnormalized_widths,
    unnormalized_heights,
    unnormalized_derivatives,
    inverse=False,
    left=0.0,
    right=1.0,
    bottom=0.0,
    top=1.0,
    min_bin_width=DEFAULT_MIN_BIN_WIDTH,
    min_bin_height=DEFAULT_MIN_BIN_HEIGHT,
    min_derivative=DEFAULT_MIN_DERIVATIVE,
):


    num_bins = unnormalized_widths.shape[-1]

    if min_bin_width * num_bins > 1.0:
        raise ValueError("Minimal bin width too large for the number of bins")
    if min_bin_height * num_bins > 1.0:
        raise ValueError("Minimal bin height too large for the number of bins")

    widths = nn.softmax(unnormalized_widths, axis=-1)
    widths = min_bin_width + (1 - min_bin_width * num_bins) * widths
    cumwidths = np.cumsum(widths, axis=-1)
    cumwidths = np.pad(
        cumwidths, [(0, 0)] * (len(cumwidths.shape) - 1) + [(1, 0)], mode="constant", constant_values=0.0
    )
    cumwidths = (right - left) * cumwidths + left
    cumwidths = cumwidths.at[..., 0].set(left)  # cumwidths[..., 0] = left
    cumwidths = cumwidths.at[..., -1].set(right)  # cumwidths[..., -1] = right
    widths = cumwidths[..., 1:] - cumwidths[..., :-1]

    derivatives = min_derivative + nn.softplus(unnormalized_derivatives)

    heights = nn.softmax(unnormalized_heights, axis=-1)
    heights = min_bin_height + (1 - min_bin_height * num_bins) * heights
    cumheights = np.cumsum(heights, axis=-1)
    cumheights = np.pad(
        cumheights, [(0, 0)] * (len(cumheights.shape) - 1) + [(1, 0)], mode="constant", constant_values=0.0
    )
    cumheights = (top - bottom) * cumheights + bottom
    cumheights = cumheights.at[..., 0].set(bottom)  # cumheights[..., 0] = bottom
    cumheights = cumheights.at[..., -1].set(top)  # cumheights[..., -1] = top
    heights = cumheights[..., 1:] - cumheights[..., :-1]

    if inverse:
        bin_idx = searchsorted(cumheights, inputs)[..., None]
    else:
        bin_idx = searchsorted(cumwidths, inputs)[..., None]

    input_cumwidths = np.take_along_axis(cumwidths, bin_idx, -1)[..., 0]  # cumwidths.gather(-1, bin_idx)[..., 0]
    input_bin_widths = np.take_along_axis(widths, bin_idx, -1)[..., 0]  # widths.gather(-1, bin_idx)[..., 0]

    input_cumheights = np.take_along_axis(cumheights, bin_idx, -1)[..., 0]  # cumheights.gather(-1, bin_idx)[..., 0]
    delta = heights / widths
    input_delta = np.take_along_axis(delta, bin_idx, -1)[..., 0]  # delta.gather(-1, bin_idx)[..., 0]

    input_derivatives = np.take_along_axis(derivatives, bin_idx, -1)[..., 0]  # derivatives.gather(-1, bin_idx)[..., 0]
    input_derivatives_plus_one = np.take_along_axis(
        derivatives[..., 1:], bin_idx, -1
    )  # derivatives[..., 1:].gather(-1, bin_idx)
    input_derivatives_plus_one = input_derivatives_plus_one[..., 0]

    input_heights = np.take_along_axis(heights, bin_idx, -1)[..., 0]  # heights.gather(-1, bin_idx)[..., 0]

    if inverse:
        a = (inputs - input_cumheights) * (
            input_derivatives + input_derivatives_plus_one - 2 * input_delta
        ) + input_heights * (input_delta - input_derivatives)
        b = input_heights * input_derivatives - (inputs - input_cumheights) * (
            input_derivatives + input_derivatives_plus_one - 2 * input_delta
        )
        c = -input_delta * (inputs - input_cumheights)

        discriminant = np.square(b) - 4 * a * c

        root = (2 * c) / (-b - np.sqrt(discriminant))
        outputs = root * input_bin_widths + input_cumwidths

        theta_one_minus_theta = root * (1 - root)
        denominator = input_delta + (
            (input_derivatives + input_derivatives_plus_one - 2 * input_delta) * theta_one_minus_theta
        )
        derivative_numerator = np.square(input_delta) * (
            input_derivatives_plus_one * np.square(root)
            + 2 * input_delta * theta_one_minus_theta
            + input_derivatives * np.square(1 - root)
        )
        logabsdet = np.log(derivative_numerator) - 2 * np.log(denominator)
        return outputs, -logabsdet
    else:
        theta = (inputs - input_cumwidths) / input_bin_widths
        theta_one_minus_theta = theta * (1 - theta)

        numerator = input_heights * (input_delta * np.square(theta) + input_derivatives * theta_one_minus_theta)
        denominator = input_delta + (
            (input_derivatives + input_derivatives_plus_one - 2 * input_delta) * theta_one_minus_theta
        )
        outputs = input_cumheights + numerator / denominator

        derivative_numerator = np.square(input_delta) * (
            input_derivatives_plus_one * np.square(theta)
            + 2 * input_delta * theta_one_minus_theta
            + input_derivatives * np.square(1 - theta)
        )
        logabsdet = np.log(derivative_numerator) - 2 * np.log(denominator)
        return outputs, logabsdet





def NeuralSpline1D(network,K=5, B=3, hidden_dim=64):
    def init_fun(rng, input_dim, **kwargs):
        params, apply_fun = network(rng,input_dim,3*K-1,hidden_dim)

        def direct_fun(params, x):
            log_det = np.zeros(x.shape[0])
            out = apply_fun(params,x[:,1:]).reshape(-1, 1,3 * K - 1)
            W, H, D = np.array_split(out, 3, axis=2)
            W, H = nn.softmax(W, axis=2), nn.softmax(H, axis=2)
            W, H = 2 * B * W, 2 * B * H
            D = nn.softplus(D)
            out, ld = unconstrained_RQS( x[:,:1], W, H, D, inverse=False, tail_bound=B)
            log_det += np.sum(ld, axis=1)
            return np.concatenate([out,x[:,1:]], axis=1), log_det.reshape((x.shape[0],))

        def inverse_fun(params, z):
            log_det = np.zeros(z.shape[0])
            out = apply_fun(params,z[:,1:]).reshape(-1, 1,3 * K - 1)
            W, H, D = np.array_split(out, 3, axis=2)
            W, H = nn.softmax(W, axis=2), nn.softmax(H, axis=2)
            W, H = 2 * B * W, 2 * B * H
            D = nn.softplus(D)
            out, ld = unconstrained_RQS( z[:,:1], W, H, D, inverse=True, tail_bound=B)
            log_det += np.sum(ld, axis=1)
            return np.concatenate([out,z[:,1:]], axis=1), log_det.reshape((z.shape[0],))

        return params, direct_fun, inverse_fun

    return init_fun


In [ ]:


def Serial(*init_funs):
    """
    Args:
        *init_funs: Multiple bijections in sequence

    Returns:
        An ``init_fun`` mapping ``(rng, input_dim)`` to a ``(params, direct_fun, inverse_fun)`` triplet.

    Examples:
        >>> num_examples, input_dim, tol = 20, 3, 1e-4
        >>> layer_rng, input_rng = random.split(random.PRNGKey(0))
        >>> inputs = random.uniform(input_rng, (num_examples, input_dim))
        >>> init_fun = Serial(Shuffle(), Shuffle())
        >>> params, direct_fun, inverse_fun = init_fun(layer_rng, input_dim)
        >>> mapped_inputs = direct_fun(params, inputs)[0]
        >>> reconstructed_inputs = inverse_fun(params, mapped_inputs)[0]
        >>> np.allclose(inputs, reconstructed_inputs).item()
        True
    """

    def init_fun(rng, conditional_dim, **kwargs):
        all_params, direct_funs, inverse_funs = [], [], []
        for init_fun in init_funs:
            rng, layer_rng = random.split(rng)
            param, direct_fun, inverse_fun = init_fun(layer_rng,conditional_dim)
            all_params.append(param)
            direct_funs.append(direct_fun)
            inverse_funs.append(inverse_fun)


        def feed_forward(params, apply_funs, inputs):
            log_det_jacobians = np.zeros(inputs.shape[0])
            
            for apply_fun, param in zip(apply_funs, params):

                inputs, log_det_jacobian = apply_fun(param,inputs,**kwargs)
                log_det_jacobians += log_det_jacobian  

            return inputs, log_det_jacobians

            
        def direct_fun(params, inputs, **kwargs):
            return feed_forward(params, direct_funs, inputs)

        def inverse_fun(params, inputs, **kwargs):
            return feed_forward(reversed(params), reversed(inverse_funs), inputs)

        return all_params, direct_fun, inverse_fun

    return init_fun


In [ ]:
from jax.scipy.stats import norm, multivariate_normal,uniform


def Normal():
    """
    Returns:
        A function mapping ``(rng, input_dim)`` to a ``(params, log_pdf, sample)`` triplet.
    """

    def init_fun(rng, input_dim):
        def log_pdf(params, inputs):
            return norm.logpdf(inputs,loc=0,scale=1).sum(1)

        def sample(rng, params, num_samples=1):
            return random.normal(rng, (num_samples, input_dim))*1

        return (), log_pdf, sample

    return init_fun

def Flow(transformation, prior=Normal()):
    """
    Args:
        transformation: a function mapping ``(rng, input_dim)`` to a
            ``(params, direct_fun, inverse_fun)`` triplet
        prior: a function mapping ``(rng, input_dim)`` to a
            ``(params, log_pdf, sample)`` triplet

    Returns:
        A function mapping ``(rng, input_dim)`` to a ``(params, log_pdf, sample)`` triplet.

    Examples:
        >>> import flows
        >>> input_dim, rng = 3, random.PRNGKey(0)
        >>> transformation = flows.Serial(
        ...     flows.Reverse(),
        ...     flows.Reverse()
        ... )
        >>> init_fun = flows.Flow(transformation, Normal())
        >>> params, log_pdf, sample = init_fun(rng, input_dim)
    """

    def init_fun(rng,conditional_dim):
        transformation_rng, prior_rng = random.split(rng)
        params, direct_fun, inverse_fun = transformation(transformation_rng,conditional_dim)
        prior_params, prior_log_pdf, prior_sample = prior(prior_rng, 1)

        def log_pdf(params, inputs):
            u, log_det = direct_fun(params, inputs)
            log_probs = prior_log_pdf(prior_params, u[:,:1])
            return log_probs + log_det
        def sample(rng, params,conditionals, num_samples=1):
            prior_samples = prior_sample(rng, prior_params, num_samples)
            x=inverse_fun(params, np.column_stack((prior_samples,conditionals)))[0]
            return x

        def log_pdf_grid(params,conditionals,search_range=1,searches=6000):
            gals=conditionals.shape[0]
            z_grid_1D=np.linspace(0,search_range,searches).reshape(1,searches)
            z_grid=np.repeat(z_grid_1D,gals,axis=0)
            prob_grid=np.zeros((gals,searches))
            for i in range(searches):

              prob_grid=prob_grid.at[:,i].set(log_pdf(params,np.column_stack((z_grid[:,i].reshape(gals,1),conditionals))))

              if i%100==0:
                print(i)


            pred=z_grid[np.arange(len(z_grid)),np.argmax(prob_grid,axis=1)]

            return z_grid_1D[0,:],prob_grid,pred

        def sample_pred(rng,params,conditionals,samples=1000):
          gals=conditionals.shape[0]
          z_samples=np.zeros((gals,samples))
          prob_grid=np.zeros((gals,samples))
          for i in range(samples):
            if i%100==0:
              print(i)
            rng,sample_rng  = random.split(rng)
            z_samples=z_samples.at[:,i].set(sample(sample_rng,params,conditionals,gals)[:,0])
            prob_grid=prob_grid.at[:,i].set(log_pdf(params,np.column_stack((z_samples[:,i].reshape(gals,1),conditionals))))

          return z_samples[np.arange(len(z_samples)),np.argmax(prob_grid,axis=1)]



        return params, log_pdf,log_pdf_grid, sample,sample_pred
    return init_fun

def network(rng,conditional_dim,out_dim, hidden_dim):
    init_fun,apply_fun=stax.serial(stax.Dense(hidden_dim), Relu, stax.Dense(1024), Relu, stax.Dense(hidden_dim), Relu, stax.Dense(out_dim),)
    _, params = init_fun(rng, (conditional_dim,))
    return params,apply_fun

In [ ]:
rng, flow_rng = random.split(random.PRNGKey(0))

init_fun = Flow(Serial(*(NeuralSpline1D(network,K=5,B=3,hidden_dim=512),)*3),prior=Normal())
params, log_pdf,log_pdf_grid,sample,sample_pred = init_fun(flow_rng,54 )

In [ ]:

opt_init, opt_update, get_params = optimizers.adam(step_size=1e-3)
opt_state = opt_init(params)

In [ ]:
from tqdm.notebook import trange
import itertools
import numpy.random as npr

In [ ]:

def loss_fn(params, inputs):
    return -log_pdf(params, inputs).mean()

@jit
def step(i, opt_state, inputs):
    params = get_params(opt_state)
    loss,gradients = value_and_grad(loss_fn)(params,inputs)
    return loss, opt_update(i, gradients, opt_state)

In [ ]:
def minmax_scale(X,min,max):
  return (X - min) / (max - min)

def minmax_unscale(X,min,max):
  return X * (max - min) + min

n_input=54
n_samples =  1000000 # just use train everything in a batch
beta=True
small_range=False
magnitudes=False
a=0
if beta:
  a+=5

x_data=onp.zeros((n_samples,n_input))
y_data=onp.zeros(n_samples)

cats=int(n_samples/1e6)

min=np.load('nsf_norm_min.npy',allow_pickle=True)
max=np.load('nsf_norm_max.npy',allow_pickle=True)

def mag_err(flux_err,flux):
    
    return 1.09*flux_err/flux

def mag(flux):
    x=-2.5*onp.log10(flux) + 23.9 
    x[onp.isnan(x)]=0
    return x 

filter_names=onp.array([])
for c in range(cats):
    cat = table.Table.read('sim/mil'+str(c+21)+'_noisy_gal.fits',format='fits',hdu=1)
    keys=cat.keys()
    count=0
    y_data[(c)*1000000:(c+1)*1000000]=cat['redshift']
    for key_name in keys:

        if key_name[len(key_name)-9:]=='BETA_FLUX':

                
            filt=key_name[:len(key_name)-10+a]
            filter_names=onp.append(filter_names,key_name[:len(key_name)-10])

            if magnitudes:

              x_data[(c)*1000000:(c+1)*1000000,count]=mag(cat[filt+'_FLUX'])
              x_data[(c)*1000000:(c+1)*1000000,count+1]=mag_err(cat[filt+'_FLUXERR'],cat[filt+'_FLUX'])
            else:
              x_data[(c)*1000000:(c+1)*1000000,count]=cat[filt+'_FLUX']
              x_data[(c)*1000000:(c+1)*1000000,count+1]=cat[filt+'_FLUXERR']
            
            count+=2
            
            
        
    print(c)


x_data = np.array(x_data.reshape(n_samples, n_input))

if small_range:
  ids=np.logical_and(y_data>=0,y_data<=2)
  y_data=y_data[ids]
  x_data=x_data[ids,:]



y_data = np.array(y_data.reshape(len(y_data), 1))

cat=0

X=minmax_scale(np.column_stack((y_data,x_data)),min,max)



In [ ]:
saved_params=np.load('NSF/1024_new_more_params.npy',allow_pickle=True)


In [ ]:
def ceildiv(a, b):
    return -(a // -b)

loss_arr=np.array([])
pred_arr=onp.zeros(len(y_data))
for i in range(ceildiv(len(y_data),50000)):
  print('batch: ',i)
  z_range,prob_grid,pred=log_pdf_grid(saved_params,X[i*50000:(i+1)*50000,1:],searches=1200)
  z_true=minmax_unscale(X[i*50000:(i+1)*50000,:1],min[0],max[0])
  pred_arr[i*50000:(i+1)*50000]=pred

In [ ]:
print(np.mean(loss_arr))

In [ ]:
def pit(prob_grid,z_grid_1D,real):
  gals=len(prob_grid)
  pit_val=onp.zeros(gals)
  z_grid=np.repeat(z_grid_1D.reshape(1,len(z_grid_1D)),gals,axis=0)
  id_grid=np.repeat(np.arange(len(z_grid_1D)).reshape(1,len(z_grid_1D)),gals,axis=0)
  
  ids=id_grid[np.arange(len(z_grid)),np.argmin(np.absolute(z_grid-real),axis=1)]
  prob=np.exp(prob_grid)/6
  for i in range(gals):
    pit_val[i]=np.trapz(prob[i,:ids[i]+1],z_grid_1D[:ids[i]+1])
    if i%1000==0:
      print(i)
      print(pit_val[i])
  return pit_val


y_pit=pit(prob_grid,z_range*6,z_true)

In [ ]:
photoz=minmax_unscale(pred_arr,min[0],max[0])



np.save('NSF_photoz.npy',np.column_stack((photoz.reshape(len(photoz),1),y_data[:,:1])),allow_pickle=True)
np.save('NSF_pit.npy',np.column_stack((y_pit.reshape(len(y_pit),1),z_true)),allow_pickle=True)


In [ ]:
photoz__=np.load('NSF_photoz.npy')
y_pit__=np.load('NSF_pit.npy')

photoz=photoz__[:,0]
y_pit=y_pit__[:,0]




In [ ]:
def sigma_nmad(pred,real):

  med=np.median(pred-real)

  return 1.48*np.median(np.absolute(pred-real-med)/(1+real))

def outlier_frac(pred,real):
  return len(pred[np.absolute((pred-real)/(1+real))>0.15])/len(pred)


def bias(pred,real):
  return np.median((pred-real)/(1+real))

def plot_func(fun,pred,real,bins,zlim,ax,color='blue',label=None):
  bin_edges=onp.linspace(0,zlim,bins+1)
  vals=onp.array([])
  for i in range(bins):
    logic= onp.logical_and(real>bin_edges[i],real<bin_edges[i+1])
    vals=onp.append(vals,fun(pred[logic],real[logic]))
  width=bin_edges[1]-bin_edges[0]

  if ax==None:
    plot=plt.plot(-width*0.5+bin_edges[1:],vals)
    plt.xlabel('Spectroscopic Redshift',fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)


  else:
    plot=ax.plot(-width*0.5+bin_edges[1:],vals,color=color,label=label)
    ax.set_xlabel('True Redshift',fontsize=20)
    for tick in ax.xaxis.get_major_ticks():
      tick.label.set_fontsize(18) 
    for tick in ax.yaxis.get_major_ticks():
      tick.label.set_fontsize(18) 
  return plot






dz=np.absolute((photoz-y_data[:1000000,0])/(1+y_data[:1000000,0]))



print('PERFORMANCE ON SYNTHETIC DATA')
print('       ')
print(r'Median delta z /(1+z) ',bias(photoz,y_data[:,0]))
print('Overall Sigma_NMAD/(1+z) ',sigma_nmad(photoz,y_data[:,0]))
print('Outlier Fraction ',outlier_frac(photoz,y_data[:,0]))
print('Median PIT: ', np.median(y_pit))
print('        ')

bins=np.linspace(0,6,100)
import matplotlib as mpl
plt.figure(figsize=(14,10))
plt.hist2d(y_data[:,0],photoz,bins=bins,cmap='hot',norm=mpl.colors.LogNorm())
plt.xlabel('True Redshift',fontsize=18)
plt.ylabel('Photometric Redshift Estimate',fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
cbar=plt.colorbar()
cbar.ax.tick_params(labelsize=16)
plt.show()

fig, axs = plt.subplots(2, 2,figsize=(16,10))
plot=plot_func(bias,photoz,y_data[:,0],20,6,ax=axs[0,0])
axs[0,0].set_ylabel(r'$<\Delta z /(1+z)>$',fontsize=20)



plot=plot_func(sigma_nmad,photoz,y_data[:,0],20,6,ax=axs[1,0])
axs[1,0].set_ylabel(r'$\sigma_{NMAD}/(1+z)$',fontsize=20)



plot=plot_func(outlier_frac,photoz,y_data[:,0],20,6,ax=axs[0,1])
axs[0,1].set_ylabel(r'$\eta_{0.15}$',fontsize=20)


axs[1,1].hist(y_pit,bins=20,edgecolor='black',color='blue')
axs[1,1].set_ylabel('Galaxies',fontsize=20)
axs[1,1].set_xlabel('PIT',fontsize=20)

for tick in axs[1,1].xaxis.get_major_ticks():
    tick.label.set_fontsize(18) 
for tick in axs[1,1].yaxis.get_major_ticks():
    tick.label.set_fontsize(18) 
fig.tight_layout()
plt.show()


In [ ]:


cat0 = table.Table.read('COSMOS2020.fits',format='fits',hdu=1)

spec_cat=cat0
n_samples=len(spec_cat)
x_spec=onp.zeros((n_samples,n_input))
keys=spec_cat.keys()
filt_names=onp.array([])
y_spec=spec_cat['ZSPEC']
count=0
for key_name in keys:
  if key_name[len(key_name)-4:]=='FLUX':
    
    filt=key_name[:len(key_name)-5]
    filt_names=onp.append(filt_names,filt)
    x_spec[:,count]=spec_cat[filt+'_FLUX']
    x_spec[:,count+1]=spec_cat[filt+'_FLUXERR']

    count+=2

spec_cat=0
cat0=0


y_spec=np.array(y_spec)
y_spec=y_spec.reshape(len(y_spec),1)
x_spec=np.array(x_spec)



In [ ]:

print(np.median(np.min(x_data[dz<0.1,:],axis=1)))
print(np.median(np.min(x_data[dz>0.1,:],axis=1)))

def mag(flux):
    x=-2.5*onp.log10(flux) + 23.9 
    x[onp.isnan(x)]=-2.5*onp.log10(-flux[onp.isnan(x)]) + 23.9 

    return x 
flux=np.array([True,False]*27)
error=np.array([False,True]*27)



bins=np.linspace(10,40,100)



import matplotlib as mpl

plt.figure(figsize=(15,10))

plt.hist(mag(np.max(x_data[dz<0.15,:][:,flux],axis=1)),density=True,color='blue',histtype='step',bins=bins,linewidth=3)
plt.hist(mag(np.max(x_data[dz>0.15,:][:,flux],axis=1)),density=True,color='red',histtype='step',bins=bins,linewidth=3)
plt.hist(mag(np.max(x_spec[:,flux],axis=1)),density=True,color='green',histtype='step',bins=bins,linewidth=3)
plt.xlabel('Brightest Band Magnitude',fontsize=16)
plt.ylabel('Frequency Density',fontsize=16)

plt.show()
neg_bins=np.linspace(-10,10,100)

for i in range(27):

  plt.figure(figsize=(15,10))
  plt.hist(mag(x_data[dz<0.22,2*i]),density=True,color='blue',histtype='step',bins=bins,linewidth=3,label=r'Sythnetic Galaxies $|\Delta z|/(1+z)<0.15$')
  plt.hist(mag(x_data[dz>0.15,2*i]),density=True,color='red',histtype='step',bins=bins,linewidth=3,label=r'Synthetic Galaxies $|\Delta z|/(1+z)>0.15$ ')
  plt.hist(mag(x_spec[:,2*i]),density=True,color='green',histtype='step',bins=bins,linewidth=3,label='COSMOS20 Galaxies')
  plt.legend(fontsize=16)
  plt.ylabel('Frequency Density',fontsize=16)
  plt.xlabel(filter_names[i]+' Magnitude',fontsize=16)
  plt.show()










In [ ]:
fig, axs = plt.subplots(2, 2,figsize=(16,10))



bins=np.linspace(15,35,50)

i=2
axs[0,0].hist(mag(x_data[dz<0.15,2*i]),density=True,color='blue',histtype='step',bins=bins,linewidth=3,label='Sythnetic Galaxies \n'r'  $ |\Delta z|/(1+z)<0.15$')
axs[0,0].hist(mag(x_data[dz>0.15,2*i]),density=True,color='red',histtype='step',bins=bins,linewidth=3,label='Synthetic Galaxies \n'r' $ |\Delta z|/(1+z)>0.15$ ')
axs[0,0].hist(mag(x_spec[:,2*i]),density=True,color='green',histtype='step',bins=bins,linewidth=3,label='COSMOS20')

axs[0,0].set_ylabel('Frequency Density',fontsize=20)
axs[0,0].set_xlabel(filter_names[i]+' magnitude',fontsize=20)


for tick in axs[0,0].xaxis.get_major_ticks():
    tick.label.set_fontsize(18) 
for tick in axs[0,0].yaxis.get_major_ticks():
    tick.label.set_fontsize(18) 

i=3
axs[1,0].hist(mag(x_data[dz<0.15,2*i]),density=True,color='blue',histtype='step',bins=bins,linewidth=3,label=r'Sythnetic Galaxies $|\Delta z|/(1+z)<0.15$')
axs[1,0].hist(mag(x_data[dz>0.15,2*i]),density=True,color='red',histtype='step',bins=bins,linewidth=3,label=r'Synthetic Galaxies $|\Delta z|/(1+z)>0.15$ ')
axs[1,0].hist(mag(x_spec[:,2*i]),density=True,color='green',histtype='step',bins=bins,linewidth=3,label='COSMOS20 Galaxies')

axs[1,0].set_ylabel('Frequency Density',fontsize=20)
axs[1,0].set_xlabel(filter_names[i]+' magnitude',fontsize=20)


for tick in axs[1,0].xaxis.get_major_ticks():
    tick.label.set_fontsize(18) 
for tick in axs[1,0].yaxis.get_major_ticks():
    tick.label.set_fontsize(18) 


i=4
axs[0,1].hist(mag(x_data[dz<0.15,2*i]),density=True,color='blue',histtype='step',bins=bins,linewidth=3,label='Sythnetic Galaxies \n'r'  $ |\Delta z|/(1+z)<0.15$')
axs[0,1].hist(mag(x_data[dz>0.15,2*i]),density=True,color='red',histtype='step',bins=bins,linewidth=3,label='Synthetic Galaxies \n'r' $ |\Delta z|/(1+z)>0.15$ ')
axs[0,1].hist(mag(x_spec[:,2*i]),density=True,color='green',histtype='step',bins=bins,linewidth=3,label='COSMOS20 Galaxies')

axs[0,1].set_ylabel('Frequency Density',fontsize=20)
axs[0,1].set_xlabel(filter_names[i]+' magnitude',fontsize=20)


for tick in axs[0,1].xaxis.get_major_ticks():
    tick.label.set_fontsize(18) 
for tick in axs[0,1].yaxis.get_major_ticks():
    tick.label.set_fontsize(18) 


i=5
axs[1,1].hist(mag(x_data[dz<0.15,2*i]),density=True,color='blue',histtype='step',bins=bins,linewidth=3,label=r'Sythnetic Galaxies $|\Delta z|/(1+z)<0.15$')
axs[1,1].hist(mag(x_data[dz>0.15,2*i]),density=True,color='red',histtype='step',bins=bins,linewidth=3,label=r'Synthetic Galaxies $|\Delta z|/(1+z)>0.15$ ')
axs[1,1].hist(mag(x_spec[:,2*i]),density=True,color='green',histtype='step',bins=bins,linewidth=3,label='COSMOS20 Galaxies')

axs[1,1].set_ylabel('Frequency Density',fontsize=20)
axs[1,1].set_xlabel(filter_names[i]+' magnitude',fontsize=20)


for tick in axs[1,1].xaxis.get_major_ticks():
    tick.label.set_fontsize(18) 
for tick in axs[1,1].yaxis.get_major_ticks():
    tick.label.set_fontsize(18) 

axs[0,0].legend(fontsize=18,loc='upper left',frameon=False)

fig.tight_layout()

plt.show()

In [ ]:
plt.hist(photoz,bins=100)
plt.show()
plt.hist(y_data[:,0],bins=100)
plt.show()

In [ ]:

n_input=54
n_samples =  10000000 # just use train everything in a batch
beta=True
small_range=False
magnitudes=False
a=0
if beta:
  a+=5

x_train_data=onp.zeros((n_samples,n_input))
y_train_data=onp.zeros(n_samples)

cats=int(n_samples/1e6)

def mag_err(flux_err,flux):
    
    return 1.09*flux_err/flux

def mag(flux):
    x=-2.5*onp.log10(flux) + 23.9 
    x[onp.isnan(x)]=0
    return x 

  
for c in range(cats):
    cat = table.Table.read('sim/mil'+str(c+1)+'_noisy_gal.fits',format='fits',hdu=1)
    keys=cat.keys()
    count=0
    y_train_data[(c)*1000000:(c+1)*1000000]=cat['redshift']
    for key_name in keys:

        if key_name[len(key_name)-9:]=='BETA_FLUX':

                
            filt=key_name[:len(key_name)-10+a]

            if magnitudes:

              x_train_data[(c)*1000000:(c+1)*1000000,count]=mag(cat[filt+'_FLUX'])
              x_train_data[(c)*1000000:(c+1)*1000000,count+1]=mag_err(cat[filt+'_FLUXERR'],cat[filt+'_FLUX'])
            else:
              x_train_data[(c)*1000000:(c+1)*1000000,count]=cat[filt+'_FLUX']
              x_train_data[(c)*1000000:(c+1)*1000000,count+1]=cat[filt+'_FLUXERR']
            
            count+=2
            
            
        
    print(c)


x_train_data = np.array(x_train_data.reshape(n_samples, n_input))


y_train_data = np.array(y_train_data.reshape(len(y_train_data), 1))


X_train=minmax_scale(np.column_stack((y_train_data,x_train_data)),min,max)




In [ ]:
z_range,prob_grid,pred=log_pdf_grid(saved_params,X[:50000,1:],searches=1200)
pred=minmax_unscale(pred,min[0],max[0])


In [ ]:
def mag(flux):
    x=-2.5*onp.log10(flux) + 23.9 
    x[onp.isnan(x)]=-2.5*onp.log10(-flux[onp.isnan(x)]) + 23.9 
    return x 
def importance_hist(test_gal,training_gals,training_redshift,ax=None):

  flux=np.array([True,False]*27)
  error=np.array([False,True]*27)

  test_mag=mag(test_gal[:,flux])
  train_mag=mag(training_gals[:,flux])
  train_error=mag_err(training_gals[:,error],test_gal[:,flux])
  

  log_weights= -np.sum(np.log(train_error),axis=1)- 0.5 *np.sum(((test_mag-train_mag)/train_error)**2,axis=1)

  frq, edges = np.histogram(training_redshift.reshape(len(training_redshift),), bins=50, weights=np.exp(log_weights), density=True)
  if ax!=None:
    the_plot=ax.bar(edges[:-1], frq, width=np.diff(edges), edgecolor="black", align="edge",color='blue')
  else:
    the_plot=plt.bar(edges[:-1], frq, width=np.diff(edges), edgecolor="black", align="edge")

  return the_plot


fig, axs = plt.subplots(3, 2,sharex=True,figsize=(20,16))
dz=np.absolute((photoz-y_data[:1000000,0])/(1+y_data[:1000000,0]))



select_id=372


axs[0,0].plot(z_range*6,np.exp(prob_grid[select_id,:])/6,color='blue',label='PDF')
axs[0,0].plot([y_data[select_id,0],y_data[select_id,0]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='red',label='True Redshift')
axs[0,0].plot([pred[select_id],pred[select_id]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='blue',linestyle='--',label='Photometric Redshift')
print(select_id)
print(dz[select_id])

select_id=41


axs[1,0].plot(z_range*6,np.exp(prob_grid[select_id,:])/6,color='blue',label='PDF')
axs[1,0].plot([y_data[select_id,0],y_data[select_id,0]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='red',label='True Redshift')
axs[1,0].plot([pred[select_id],pred[select_id]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='blue',linestyle='--',label='Photometric Redshift')
print(select_id)
print(dz[select_id])

select_id=594


axs[2,0].plot(z_range*6,np.exp(prob_grid[select_id,:])/6,color='blue',label='PDF')
axs[2,0].plot([y_data[select_id,0],y_data[select_id,0]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='red',label='True Redshift')
axs[2,0].plot([pred[select_id],pred[select_id]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='blue',linestyle='--',label='Photometric Redshift')
print(select_id)
print(dz[select_id])

select_id=627

axs[0,1].plot(z_range*6,np.exp(prob_grid[select_id,:])/6,color='blue',label='PDF')
axs[0,1].plot([y_data[select_id,0],y_data[select_id,0]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='red',label='True Redshift')
axs[0,1].plot([pred[select_id],pred[select_id]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='blue',linestyle='--',label='Photometric Redshift')
print(select_id)
print(dz[select_id])

select_id=4007


axs[1,1].plot(z_range*6,np.exp(prob_grid[select_id,:])/6,color='blue',label='PDF')
axs[1,1].plot([y_data[select_id,0],y_data[select_id,0]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='red',label='True Redshift')
axs[1,1].plot([pred[select_id],pred[select_id]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='blue',linestyle='--',label='Photometric Redshift')
print(select_id)
print(dz[select_id])

select_id=3573


axs[2,1].plot(z_range*6,np.exp(prob_grid[select_id,:])/6,color='blue',label='PDF')
axs[2,1].plot([y_data[select_id,0],y_data[select_id,0]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='red',label='True Redshift')
axs[2,1].plot([pred[select_id],pred[select_id]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='blue',linestyle='--',label='Photometric Redshift')
print(select_id)
print(dz[select_id])





fig.subplots_adjust(hspace=0)



axs[2,0].set_xlabel(r'Redshift $z$',fontsize=20)
axs[0,0].set_ylabel(r'$p(z|x)$',fontsize=20)
axs[1,0].set_ylabel(r'$p(z|x)$',fontsize=20)
axs[2,0].set_ylabel(r'$p(z|x)$',fontsize=20)

axs[2,1].set_xlabel(r'Redshift $z$',fontsize=20)
axs[0,1].set_ylabel(r'$p(z|x)$',fontsize=20)
axs[1,1].set_ylabel(r'$p(z|x)$',fontsize=20)
axs[2,1].set_ylabel(r'$p(z|x)$',fontsize=20)



for tick in axs[2,0].xaxis.get_major_ticks():
    tick.label.set_fontsize(18) 
for tick in axs[2,1].xaxis.get_major_ticks():
    tick.label.set_fontsize(18) 

for tick in axs[0,0].yaxis.get_major_ticks():
    tick.label.set_fontsize(18) 
for tick in axs[1,0].yaxis.get_major_ticks():
    tick.label.set_fontsize(18) 
for tick in axs[2,0].yaxis.get_major_ticks():
    tick.label.set_fontsize(18) 

for tick in axs[0,1].yaxis.get_major_ticks():
    tick.label.set_fontsize(18) 
for tick in axs[1,1].yaxis.get_major_ticks():
    tick.label.set_fontsize(18) 
for tick in axs[2,1].yaxis.get_major_ticks():
    tick.label.set_fontsize(18) 

axs[0,0].legend(fontsize=18,frameon=False)

plt.show()



In [ ]:
def minmax_scale(X,min,max):
  return (X - min) / (max - min)

def minmax_unscale(X,min,max):
  return X * (max - min) + min


x_data=0
y_data=0
cat0 = table.Table.read('COSMOS2020.fits',format='fits',hdu=1)
spec_cat=cat0[cat0['ZSPEC']!=-1]
n_samples=len(spec_cat)
x_spec=onp.zeros((n_samples,n_input))
keys=spec_cat.keys()

y_spec=spec_cat['ZSPEC']
count=0
for key_name in keys:
  if key_name[len(key_name)-4:]=='FLUX':
        
    filt=key_name[:len(key_name)-5]
    x_spec[:,count]=spec_cat[filt+'_FLUX']
    x_spec[:,count+1]=spec_cat[filt+'_FLUXERR']
    count+=2

spec_cat=0
cat0=0


x_spec=np.array(x_spec)
y_spec=np.array(y_spec)
y_spec=y_spec.reshape(len(y_spec),1)
min=np.load('nsf_norm_min.npy',allow_pickle=True)
max=np.load('nsf_norm_max.npy',allow_pickle=True)

In [ ]:
!pip install jaxopt

In [ ]:
from tqdm.notebook import trange
import itertools
import numpy.random as npr

from numpy.ma.core import add
from jaxopt import ScipyBoundedMinimize as minimize


x_spec = np.array(x_spec.reshape(n_samples, n_input))
y_spec = np.array(y_spec.reshape(n_samples, 1))

params=saved_params
alpha_mask=onp.identity(54)
beta_mask=onp.identity(54)

one_col= onp.zeros((len(x_spec),54))
two_col= onp.zeros((len(x_spec),54))
x_spec_flux=onp.zeros((len(x_spec),54))

for i in range(27):
  alpha_mask[2*i+1,2*i+1]=0
  beta_mask[2*i,2*i]=0
  one_col[:,2*i]=1
  two_col[:,2*i+1]=1
  x_spec_flux[:,2*i+1]=copy.deepcopy(x_spec[:,2*i])


alpha_mask=np.array(alpha_mask)
beta_mask=np.array(beta_mask)
one_col=np.array(one_col)
two_col=np.array(two_col)
x_spec_flux=np.array(x_spec_flux)

def add_cal(p0,x_data):
  the_params_1d=p0.reshape(1,54)

  the_params= np.repeat(the_params_1d, repeats=len(x_spec), axis=0)

  ones=np.ones(len(x_spec)).reshape(len(x_spec),1)
  params_shift=np.column_stack((ones,the_params))
  params_shift=np.delete(params_shift,-1,axis=1)

  
  alpha_flux=np.dot(the_params,alpha_mask)+two_col
  beta_err=np.dot(the_params,beta_mask)
  alpha_err=np.dot(params_shift,beta_mask) +one_col


  x_corr=copy.deepcopy(x_spec)
  x_corr= ((x_corr*alpha_err)**2 + (beta_err*alpha_err*x_spec_flux)**2)**0.5

  x_corr=x_corr*alpha_flux


  return x_corr
  
def opt_func(opt_params):
  

  loss_val=loss_fn(params, minmax_scale(np.column_stack((y_spec,add_cal(opt_params,x_spec))),min,max))
  return loss_val


alpha_bound0=[0.8,1.2]
beta_bound0=[0.01,0.01]

lower_bounds=onp.array([])
upper_bounds=onp.array([])
for i in range(27):

  if i != 4:

    lower_bounds=onp.append(lower_bounds,alpha_bound0[0])
    lower_bounds=onp.append(lower_bounds,beta_bound0[0])

    upper_bounds=onp.append(upper_bounds,alpha_bound0[1])
    upper_bounds=onp.append(upper_bounds,beta_bound0[1])

  else:

    lower_bounds=onp.append(lower_bounds,1.0)
    lower_bounds=onp.append(lower_bounds,beta_bound0[0])

    upper_bounds=onp.append(upper_bounds,1.0)
    upper_bounds=onp.append(upper_bounds,beta_bound0[1])


bounds = (np.array(lower_bounds), np.array(upper_bounds))






best_score=loss_fn(saved_params, minmax_scale(np.column_stack((y_spec,x_spec)),min,max))
for i in range(50):

  p0=np.array([])
  for k in range(27):

    if k!=4:
      p0=np.append(p0,0.8+onp.random.random()*0.4)
    else:
      p0=np.append(p0,1.0)
    p0=np.append(p0,0.01)

  
  p0=p0.reshape(54,)

  tol = 1e-4
  method = 'SLSQP' #'L-BFGS-B' 
  options = {'disp':False,'ftol':tol, 'gtol':tol}
  opt = minimize(fun=opt_func,method=method, tol=tol,options=options,maxiter=100000)
  opt_params,_=opt.run(p0,bounds=bounds)
  #score= loss_fn(params, add_cal(opt_params,x_spec), y_spec)
  score=opt_func(opt_params)
  if score<best_score:
    best_params=copy.deepcopy(opt_params)
    best_score=score
    print(score)


p0=onp.array(best_params)

beta_bound0=[0,0.1]

lower_bounds=onp.array([])
upper_bounds=onp.array([])
for i in range(27):

  if i != 4:

    lower_bounds=onp.append(lower_bounds,alpha_bound0[0])
    lower_bounds=onp.append(lower_bounds,beta_bound0[0])

    upper_bounds=onp.append(upper_bounds,alpha_bound0[1])
    upper_bounds=onp.append(upper_bounds,beta_bound0[1])

  else:

    lower_bounds=onp.append(lower_bounds,1.0)
    lower_bounds=onp.append(lower_bounds,beta_bound0[0])

    upper_bounds=onp.append(upper_bounds,1.0)
    upper_bounds=onp.append(upper_bounds,beta_bound0[1])


bounds = (np.array(lower_bounds), np.array(upper_bounds))

for k in range(50):
  for i in range(27):
    p0[2*i+1]=onp.random.random()*0.1
  tol = 1e-4
  method = 'SLSQP' #'L-BFGS-B' 
  options = {'disp':False,'ftol':tol, 'gtol':tol}
  opt = minimize(fun=opt_func,method=method, tol=tol,options=options,maxiter=1000000)
  opt_params,_=opt.run(np.array(p0),bounds=bounds)
  score=opt_func(opt_params)
  #score= loss_fn(params, add_cal(best_params,x_spec), y_spec)
  if score<best_score:
    best_params=copy.deepcopy(opt_params)
    best_score=score
    print(score)



In [ ]:
best_params=np.load('nsf_best_calib.npy',allow_pickle=True)
print(best_params)

In [ ]:
def plot_func(fun,pred,real,bins,zlim,ax,color='blue',label=None):
  bin_edges=onp.linspace(0,zlim,bins+1)
  vals=onp.array([])
  for i in range(bins):
    logic= onp.logical_and(real>bin_edges[i],real<bin_edges[i+1])
    vals=onp.append(vals,fun(pred[logic],real[logic]))
  width=bin_edges[1]-bin_edges[0]

  if ax==None:
    plot=plt.plot(-width*0.5+bin_edges[1:],vals)
    plt.xlabel('Spectroscopic Redshift',fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)


  else:
    plot=ax.plot(-width*0.5+bin_edges[1:],vals,color=color,label=label)
    ax.set_xlabel('True Redshift',fontsize=20)
    for tick in ax.xaxis.get_major_ticks():
      tick.label.set_fontsize(18) 
    for tick in ax.yaxis.get_major_ticks():
      tick.label.set_fontsize(18) 
  return plot



def pit(prob_grid,z_grid_1D,real):
  gals=len(prob_grid)
  pit_val=onp.zeros(gals)
  z_grid=np.repeat(z_grid_1D.reshape(1,len(z_grid_1D)),gals,axis=0)
  id_grid=np.repeat(np.arange(len(z_grid_1D)).reshape(1,len(z_grid_1D)),gals,axis=0)
  
  ids=id_grid[np.arange(len(z_grid)),np.argmin(np.absolute(z_grid-real),axis=1)]
  prob=np.exp(prob_grid)/6
  for i in range(gals):
    pit_val[i]=np.trapz(prob[i,:ids[i]+1],z_grid_1D[:ids[i]+1])
    if i%1000==0:
      print(i)
      print(pit_val[i])
  return pit_val

def minmax_scale(X,min,max):
  return (X - min) / (max - min)



In [ ]:

X_spec=minmax_scale(np.column_stack((y_spec,x_spec)),min,max)

z_range,prob_grid,pred=log_pdf_grid(saved_params,X_spec[:,1:],searches=1200)
y_pred_before=minmax_unscale(pred,min[0],max[0])
y_pit_before=pit(prob_grid,z_range*6,y_spec)


X_spec=minmax_scale(np.column_stack((y_spec,add_cal(best_params,x_spec))),min,max)

z_range,prob_grid,pred=log_pdf_grid(saved_params,X_spec[:,1:],searches=1200)

y_pred=minmax_unscale(pred,min[0],max[0])
y_pit=pit(prob_grid,z_range*6,y_spec)


print('RESULTS BEFORE CALLIBRATION')
print('         ')
print(r'Median delta z /(1+z) ',bias(y_pred_before,y_spec[:,0]))
print('Overall Sigma_NMAD/(1+z) ',sigma_nmad(y_pred_before,y_spec[:,0]))
print('Outlier Fraction ',outlier_frac(y_pred_before,y_spec[:,0]))
print('Median PIT: ', np.median(y_pit_before))
print('        ')




print('RESULTS AFTER CALLIBRATION')
print('         ')
print(r'Median delta z /(1+z) ',bias(y_pred,y_spec[:,0]))
print('Overall Sigma_NMAD/(1+z) ',sigma_nmad(y_pred,y_spec[:,0]))
print('Outlier Fraction ',outlier_frac(y_pred,y_spec[:,0]))
print('Median PIT: ', np.median(y_pit))
print('        ')

bins=np.linspace(0,6,100)
import matplotlib as mpl
plt.figure(figsize=(14,10))
plt.hist2d(y_spec[:,0],y_pred,bins=bins,cmap='hot',norm=mpl.colors.LogNorm())
plt.xlabel('Spectroscopic Redshift',fontsize=18)
plt.ylabel('Photometric Redshift Estimate',fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
cbar=plt.colorbar()
cbar.ax.tick_params(labelsize=16)
plt.show()

fig, axs = plt.subplots(2, 2,figsize=(16,10))
plot=plot_func(bias,y_pred_before,y_spec[:,0],20,6,ax=axs[0,0],color='red',label='Before Calibration')
plot=plot_func(bias,y_pred,y_spec[:,0],20,6,ax=axs[0,0],color='blue',label='After Calibration')
axs[0,0].set_ylabel(r'$<\Delta z /(1+z)>$',fontsize=20)

plot=plot_func(sigma_nmad,y_pred_before,y_spec[:,0],20,6,ax=axs[1,0],color='red',label='Before Calibration')
plot=plot_func(sigma_nmad,y_pred,y_spec[:,0],20,6,ax=axs[1,0],color='blue',label='After Calibration')
axs[1,0].set_ylabel(r'$\sigma_{NMAD}/(1+z)$',fontsize=20)
axs[1,0].legend(fontsize=20)



plot=plot_func(outlier_frac,y_pred_before,y_spec[:,0],20,6,ax=axs[0,1],color='red')
plot=plot_func(outlier_frac,y_pred,y_spec[:,0],20,6,ax=axs[0,1],color='blue')
axs[0,1].set_ylabel(r'$\eta_{0.15}$',fontsize=20)

axs[1,1].hist(y_pit_before,bins=20,color='red',histtype='step',linewidth=2)
axs[1,1].hist(y_pit,bins=20,color='blue',histtype='step',linewidth=2)
axs[1,1].set_ylabel('Galaxies',fontsize=20)
axs[1,1].set_xlabel('PIT',fontsize=20)
axs[1,1].set_ylim(100,10000)
axs[1,1].set_yscale('log')
for tick in axs[1,1].xaxis.get_major_ticks():
    tick.label.set_fontsize(18) 
for tick in axs[1,1].yaxis.get_major_ticks():
    tick.label.set_fontsize(18) 
fig.tight_layout()
plt.show()


In [ ]:
import copy 
pred=copy.deepcopy(y_pred)
fig, axs = plt.subplots(2, 2,sharex=True,figsize=(16,8))



select_id=1912

axs[0,0].plot(z_range*6,np.exp(prob_grid[select_id,:])/6,color='blue',label='PDF')
axs[0,0].plot([y_spec[select_id,0],y_spec[select_id,0]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='red',label='Spectroscopic Redshift')
axs[0,0].plot([pred[select_id],pred[select_id]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='blue',linestyle='--',label='Photometric Redshift')
print(select_id)
print(dz[select_id])

select_id=8537


axs[1,0].plot(z_range*6,np.exp(prob_grid[select_id,:])/6,color='blue',label='PDF')
axs[1,0].plot([y_spec[select_id,0],y_spec[select_id,0]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='red',label='True Redshift')
axs[1,0].plot([pred[select_id],pred[select_id]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='blue',linestyle='--',label='Photometric Redshift')
print(select_id)
print(dz[select_id])



select_id=345

axs[0,1].plot(z_range*6,np.exp(prob_grid[select_id,:])/6,color='blue',label='PDF')
axs[0,1].plot([y_spec[select_id,0],y_spec[select_id,0]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='red',label='True Redshift')
axs[0,1].plot([pred[select_id],pred[select_id]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='blue',linestyle='--',label='Photometric Redshift')
print(select_id)
print(dz[select_id])

select_id=972


axs[1,1].plot(z_range*6,np.exp(prob_grid[select_id,:])/6,color='blue',label='PDF')
axs[1,1].plot([y_spec[select_id,0],y_spec[select_id,0]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='red',label='True Redshift')
axs[1,1].plot([pred[select_id],pred[select_id]],[0,np.max(np.exp(prob_grid[select_id,:])/6)],color='blue',linestyle='--',label='Photometric Redshift')
print(select_id)
print(dz[select_id])






fig.subplots_adjust(hspace=0)



axs[1,0].set_xlabel(r'Redshift $z$',fontsize=20)
axs[0,0].set_ylabel(r'$p(z|x)$',fontsize=20)
axs[1,0].set_ylabel(r'$p(z|x)$',fontsize=20)

axs[1,1].set_xlabel(r'Redshift $z$',fontsize=20)
axs[0,1].set_ylabel(r'$p(z|x)$',fontsize=20)
axs[1,1].set_ylabel(r'$p(z|x)$',fontsize=20)



for tick in axs[1,0].xaxis.get_major_ticks():
    tick.label.set_fontsize(18) 
for tick in axs[1,1].xaxis.get_major_ticks():
    tick.label.set_fontsize(18) 

for tick in axs[0,0].yaxis.get_major_ticks():
    tick.label.set_fontsize(18) 
for tick in axs[1,0].yaxis.get_major_ticks():
    tick.label.set_fontsize(18) 

for tick in axs[0,1].yaxis.get_major_ticks():
    tick.label.set_fontsize(18) 
for tick in axs[1,1].yaxis.get_major_ticks():
    tick.label.set_fontsize(18) 

axs[0,0].legend(loc='upper left',fontsize=17.5,frameon=False)

plt.show()



In [ ]:
min=np.load('nsf_norm_min.npy',allow_pickle=True)
max=np.load('nsf_norm_max.npy',allow_pickle=True)

cat0 = table.Table.read('COSMOS2020.fits',format='fits',hdu=1)
cat0 = cat0[cat0['lp_zq']>0]
n_samples=len(cat0)
x_data=onp.zeros((n_samples,n_input))
keys=cat0.keys()
filt_names=onp.array([])
count=0
for key_name in keys:
  if key_name[len(key_name)-4:]=='FLUX':
    
    filt=key_name[:len(key_name)-5]
    filt_names=onp.append(filt_names,filt)
    x_data[:,count]=cat0[filt+'_FLUX']
    x_data[:,count+1]=cat0[filt+'_FLUXERR']

    count+=2


In [ ]:
best_params=np.load('nsf_best_calib.npy',allow_pickle=True)
def add_cal(p0,x_data):
  n_samples=len(x_data)

  x_data = np.array(x_data.reshape(n_samples, n_input))

  params=saved_params
  alpha_mask=onp.identity(54)
  beta_mask=onp.identity(54)

  one_col= onp.zeros((len(x_data),54))
  two_col= onp.zeros((len(x_data),54))
  x_data_flux=onp.zeros((len(x_data),54))

  for i in range(27):
    alpha_mask[2*i+1,2*i+1]=0
    beta_mask[2*i,2*i]=0
    one_col[:,2*i]=1
    two_col[:,2*i+1]=1
    x_data_flux[:,2*i+1]=copy.deepcopy(x_data[:,2*i])


  alpha_mask=np.array(alpha_mask)
  beta_mask=np.array(beta_mask)
  one_col=np.array(one_col)
  two_col=np.array(two_col)
  x_data_flux=np.array(x_data_flux)
  the_params_1d=p0.reshape(1,54)

  the_params= np.repeat(the_params_1d, repeats=len(x_data), axis=0)

  ones=np.ones(len(x_data)).reshape(len(x_data),1)
  params_shift=np.column_stack((ones,the_params))
  params_shift=np.delete(params_shift,-1,axis=1)

  
  alpha_flux=np.dot(the_params,alpha_mask)+two_col
  beta_err=np.dot(the_params,beta_mask)
  alpha_err=np.dot(params_shift,beta_mask) +one_col


  x_corr=copy.deepcopy(x_data)
  x_corr= ((x_corr*alpha_err)**2 + (beta_err*alpha_err*x_data_flux)**2)**0.5

  x_corr=x_corr*alpha_flux


  return x_corr

In [ ]:
def ceildiv(a, b):
    return -(a // -b)

loss_arr=np.array([])
pred_arr=onp.zeros(len(x_data))
for i in range(ceildiv(len(x_data),50000)):
  print('batch: ',i)
  X_batch=minmax_scale(add_cal(best_params,x_data[i*50000:(i+1)*50000,:]),min[1:],max[1:])
  z_range,prob_grid,pred=log_pdf_grid(saved_params,X_batch,searches=1200)
  pred_arr[i*50000:(i+1)*50000]=pred
  if i==0:
    np.save('nsf_phot_pdf.npy',np.exp(prob_grid)/6,allow_pickle=True)

In [ ]:
y_pred=minmax_unscale(pred_arr,min[0],max[0])
np.save('nsf_phot_pred.npy',y_pred,allow_pickle=True)
plt.hist(y_pred,bins=1000)
plt.show()